In [10]:
import json
import os
import pandas as pd
from tqdm import tqdm
import re
import tensorflow_hub as hub
from tqdm import tqdm
from sklearn.cluster import DBSCAN
import numpy as np

In [2]:
STORIES_DIRECTORY = "data/stories/"

In [3]:
def as_nodes(data):
    def parse(node, nodes):
        nodes.append({"content": node["content"], "title": node["title"]})
        for action in node["actions"]:
            if action["subgraph"]:
                parse(action["subgraph"], nodes)
    
    nodes = []
    parse(data, nodes)
    return nodes

In [4]:
sentences = set()
titles = {}  
for file in os.listdir(STORIES_DIRECTORY):
    if not file.endswith(".json"):
        continue
    with open(os.path.join(STORIES_DIRECTORY, file), "r") as src:
        data = json.load(src)
    nodes = as_nodes(data)
    for node in nodes:
        titles[node["title"].strip()] = titles.get(node["title"].strip(), 0) + 1
        sentences |= {sent.strip()
                      for sent in re.split("[.!?]", node["content"])}

In [5]:
len(titles), len(sentences)

(32112, 676699)

In [6]:
if "" in titles:
    del titles[""]
sentences = sentences - {""}

In [7]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

2021-10-21 00:19:43.925170: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-21 00:19:43.925638: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-21 00:19:43.925677: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-10-21 00:19:43.925711: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-10-21 00:19:43.927084: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [8]:
def filteralnum(text):
    return "".join([
        char
        for char in text
        if char.isalnum()
    ])


embeddings = {}
for text in tqdm(sorted(sentences)):
    if filteralnum(text) != "":
        embeddings[text] = embed([text])[0].numpy()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 676698/676698 [16:08<00:00, 698.71it/s]


In [11]:
embeddings_np = []
for text in tqdm(sorted(sentences)):
    if filteralnum(text) != "":
        embeddings_np.append(embeddings[text].tolist())
embeddings_np = np.array(embeddings_np)
embeddings_np.shape

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 676698/676698 [00:16<00:00, 41443.45it/s]


(676632, 512)

In [13]:
clusters = DBSCAN().fit_predict(embeddings_np)

In [14]:
len(clusters)

676632

In [15]:
len(clusters[clusters != -1])

12764

In [16]:
texts = []
for text in tqdm(sorted(sentences)):
    if filteralnum(text) != "":
        texts.append(text)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 676698/676698 [00:02<00:00, 271075.25it/s]


In [17]:
df = pd.DataFrame({"text": texts, "cluster": clusters})
df.head()

,text,cluster
0,"""\n"" 'Ours is not to reason why, ours is but t...",-1
1,"""\n"" A military starship",-1
2,"""\n"" AAAGGH",-1
3,"""\n"" AGGHH",-1
4,"""\n"" Aaaaah, sweet revenge",-1


In [30]:
cluster_sizes = df.loc[df["cluster"] != -1, "cluster"].value_counts()
cluster_sizes

978     853
1004    122
14      118
121     107
275     102
       ... 
843       5
1221      4
1022      4
1180      3
736       1
Name: cluster, Length: 1256, dtype: int64

In [21]:
pd.set_option("display.max_colwidth", 256)

In [32]:
drop_clusters = []
for cluster in cluster_sizes.index:
    print("")
    print("-" * 80)
    print("")
    display(df.loc[df["cluster"] == cluster])
    print("Drop ([y]es/[n]o/[b]reak)")
    response = input().strip()
    if response == "y":
        drop_clusters.append(cluster)
    elif response == "b":
        break


--------------------------------------------------------------------------------



,text,cluster
162715,Combat: 0/7 Life: 0/7 Coins: 0/100 Page Items Links Inventory,978
162716,Combat: 0/7 Life: 7/7 Coins: 10/100 Page Items Links Inventory,978
162717,Combat: 0/7 Life: 7/7 Coins: 11/100 Page Items Links Inventory,978
162718,Combat: 0/7 Life: 7/7 Coins: 12/100 Page Items Links Inventory,978
162719,Combat: 0/7 Life: 7/7 Coins: 13/100 Page Items Links Inventory,978
...,...,...
183080,Fight - 8 Combat: 2/7 Life: 4/7 Coins: 3/100 Page Items Links Inventory,978
183081,Fight - 8 Combat: 2/7 Life: 4/7 Coins: 4/100 Page Items Links Inventory,978
183082,Fight - 8 Combat: 2/7 Life: 5/7 Coins: 4/100 Page Items Links Inventory,978
183083,Fight - 8 Combat: 2/7 Life: 6/7 Coins: 3/100 Page Items Links Inventory,978


Drop ([y]es/[n]o/[b]reak)


 y



--------------------------------------------------------------------------------



...

Drop ([y]es/[n]o/[b]reak)



--------------------------------------------------------------------------------



,text,cluster
38711,""" She said",410
38759,""" She says",410
48545,""" she said",410
48570,""" she says",410
51793,""", she says",410
79201,"""She says",410
97635,' She says,410
97889,' she says,410
357052,She says,410
357156,"She says,",410


Drop ([y]es/[n]o/[b]reak)


 b


In [34]:
df.loc[df["cluster"].isin(drop_clusters)].sort_values("cluster").groupby("cluster").head(3).reset_index(drop=False)

,index,text,cluster
0,24499,""" What do you do: Page Items Links Inventory",51
1,605730,you: Page Items Links Inventory,51
2,605160,you can: Page Items Links Inventory,51
3,366252,So what do we do now,274
4,471669,What are we going to do now,274
5,472099,What do we do now,274
6,472191,What do you plan to do about this,275
7,472190,What do you plan to do,275
8,472160,What do you intend to do with me,275
9,648921,” Page Items Links Inventory,345


In [36]:
texts = []
embeddings_np = []
for text in tqdm(sorted(titles)):
    if filteralnum(text) != "":
        texts.append(text)
        embeddings_np.append(embed([text])[0].numpy())
embeddings_np = np.array(embeddings_np)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32111/32111 [00:49<00:00, 648.42it/s]


In [37]:
clusters = DBSCAN().fit_predict(embeddings_np)

In [38]:
df = pd.DataFrame({"text": texts, "cluster": clusters})
df.head()

,text,cluster
0,""" I will let myself go, Be dramatic, and be a new Marylin.""",-1
1,""" I would try to call emergencies from my phone!""",-1
2,"""...A Shield!""",-1
3,"""...A Weapon!""",-1
4,"""...I don't believe in God...""",-1


In [39]:
cluster_sizes = df.loc[df["cluster"] != -1, "cluster"].value_counts()
cluster_sizes

19     34
23     33
89     28
77     21
34     21
       ..
86      5
36      5
32      5
122     5
0       5
Name: cluster, Length: 124, dtype: int64

In [41]:
drop_clusters = []
for cluster in cluster_sizes.index:
    print("")
    print("-" * 80)
    print("")
    display(df.loc[df["cluster"] == cluster])
    print("Drop ([y]es/[n]o/[b]reak)")
    response = input().strip()
    if response == "y":
        drop_clusters.append(cluster)
    elif response == "b":
        break


--------------------------------------------------------------------------------



,text,cluster
1122,*A few days later*,19
1964,A Few Days Later...,19
2167,A Week Later,19
2168,A Week Later...,19
2176,A Year Later...,19
2204,A couple years later...,19
2235,A few days later,19
2236,A few days later...,19
2240,A few months later,19
2241,A few months later...,19


Drop ([y]es/[n]o/[b]reak)



--------------------------------------------------------------------------------



,text,cluster
1327,"05.00AM, 14 June, 1200BC: Trojan Beach, Trojan Coast, Troy",23
1328,"06.00AM, 14 June, 1200BC: Trojan Beach, Trojan Coast, Troy",23
1329,"06.00AM, 16 June, 1200BC: Trojan Beach, Trojan Coast, Troy",23
1331,"08.00AM, 13 June, 1200BC: Trojan Beach, Trojan Coast, Troy",23
1332,"08.00AM, 14 June, 1200BC: Trojan Beach, Trojan Coast, Troy",23
1342,"09.00AM, 13 June, 1200BC: Trojan Plains, Trojan Coast, Troy",23
1343,"09.00AM, 14 June, 1200BC: Trojan Beach, Trojan Coast, Troy",23
1387,"10.00AM, 11 June, 1200BC: Trojan Beach, Trojan Coast, Troy",23
1388,"10.00AM, 13 June, 1200BC: Trojan Plains, Trojan Coast, Troy",23
1389,"10.00AM, 14 June, 1200BC: Trojan Beach, Trojan Coast, Troy",23


...

Drop ([y]es/[n]o/[b]reak)



--------------------------------------------------------------------------------



,text,cluster
15252,Just stay the night,98
23578,Stay for the Night?,98
23580,Stay for the night,98
23694,Stay the Night,98
23696,Stay the night,98


Drop ([y]es/[n]o/[b]reak)


 b


In [94]:
import re

In [95]:
re.search("[a-zA-Z0-9#., ]+\{([a-zA-Z-]\s*\:\s*[a-zA-Z0-9 ]+;\s*)+\}", 
          "Bad Ending #2 body {background-color: #FFF; }\r\np {\r\n text-indent: 50px;\r\n} \r\n#about {\r\n background-color: #F8F8F8;\r\n padding: 4px;\r\n border: 4px double red;\r\n}\r\nli {\r\n list-style-type: none;\r\n font-size: medium;} Test")

In [96]:
re.search("(([a-zA-Z0-9\#.]|(\s*,\s*))+\s*\{(\s*[a-zA-Z-]+\s*\:\s*[\!a-zA-Z0-9\# ]+\s*;)*\s*\}\s*)+", 
          "Bad Ending #2 body {background-color: #FFF; }\r\np {\r\n text-indent: 50px;\r\n} \r\n#about {\r\n background-color: #F8F8F8;\r\n padding: 4px;\r\n border: 4px double red;\r\n}\r\nli {\r\n list-style-type: none;\r\n font-size: medium;} Test")

<re.Match object; span=(14, 213), match='body {background-color: #FFF; }\r\np {\r\n text-i>

In [97]:
"Bad Ending #2 body {background-color: #FFF; }\r\np {\r\n text-indent: 50px;\r\n} \r\n#about {\r\n background-color: #F8F8F8;\r\n padding: 4px;\r\n border: 4px double red;\r\n}\r\nli {\r\n list-style-type: none;\r\n font-size: medium;} Test"\
    [14:213]

'body {background-color: #FFF; }\r\np {\r\n text-indent: 50px;\r\n} \r\n#about {\r\n background-color: #F8F8F8;\r\n padding: 4px;\r\n border: 4px double red;\r\n}\r\nli {\r\n list-style-type: none;\r\n font-size: medium;} '

In [98]:
re.search("(([a-zA-Z0-9\#.]|(\s*,\s*))+\s*\{(\s*[a-zA-Z-]+\s*\:\s*[\!a-zA-Z0-9\# ]+\s*;)*\s*\}\s*)+", 
          "Bad Ending #2 body {background-color: #FFF; }\r\np {\r\n text-indent: 50px;\r\n} \r\n#about {\r\n background-color: #F8F8F8;\r\n padding: 4px;\r\n border: 4px double red;\r\n}\r\nli {\r\n list-style-type: none;\r\n font-size: medium;} body {fond-size: 1.5em; widthL: 100%} Test")

<re.Match object; span=(14, 213), match='body {background-color: #FFF; }\r\np {\r\n text-i>

In [102]:
re.search("(([a-zA-Z0-9\#.]|(\s*,\s*))+\s*\{(\s*[a-zA-Z-]+\s*\:\s*[\!a-zA-Z0-9\#%. ]+\s*;)*\s*\}\s*)+", 
          "Bad Ending #2 body {background-color: #FFF; }\r\np {\r\n text-indent: 50px;\r\n} \r\n#about {\r\n background-color: #F8F8F8;\r\n padding: 4px;\r\n border: 4px double red;\r\n}\r\nli {\r\n list-style-type: none;\r\n font-size: medium;} body {fond-size: 1.5em; widthL: 100%;} Test")

<re.Match object; span=(14, 252), match='body {background-color: #FFF; }\r\np {\r\n text-i>

In [103]:
"Bad Ending #2 body {background-color: #FFF; }\r\np {\r\n text-indent: 50px;\r\n} \r\n#about {\r\n background-color: #F8F8F8;\r\n padding: 4px;\r\n border: 4px double red;\r\n}\r\nli {\r\n list-style-type: none;\r\n font-size: medium;} body {fond-size: 1.5em; widthL: 100%;} Test"\
    [14:252]

'body {background-color: #FFF; }\r\np {\r\n text-indent: 50px;\r\n} \r\n#about {\r\n background-color: #F8F8F8;\r\n padding: 4px;\r\n border: 4px double red;\r\n}\r\nli {\r\n list-style-type: none;\r\n font-size: medium;} body {fond-size: 1.5em; widthL: 100%;} '